In [1]:
from ultralytics import YOLO
import pandas as pd
import os

In [2]:
yolo_model = YOLO('best.pt')

In [12]:
import albumentations as A
import utilities
import cv2
import torch
import torch.nn as nn
import transformers
from PIL import Image
from torchvision import transforms
import open_clip
import torch.nn.functional as F
import glob
from tqdm import tqdm

In [4]:
class Head(nn.Module):
    def __init__(self, hidden_size):
        super(Head, self).__init__()

        self.emb = nn.Linear(hidden_size, 512, bias=False)
        self.arc = utilities.ArcMarginProduct_subcenter(512, 5)
        self.dropout = utilities.Multisample_Dropout()

    def forward(self, x):
        embeddings = self.dropout(x, self.emb)
        
        output = self.arc(embeddings)

        return output, F.normalize(embeddings)

In [5]:
class Model(nn.Module):
    def __init__(self, vit_backbone):
        super(Model, self).__init__()

        self.vit_backbone = vit_backbone
        with torch.no_grad():
            emb_size = vit_backbone(torch.randn(2, 3, image_size, image_size))[0].shape[1]
        self.head = Head(emb_size)

    def forward(self, images):
        x = self.vit_backbone.encode_image(images)
        
        return self.head(x)


In [6]:
model_name = 'ViT-H-14-378-quickgelu'
pretrained = 'dfn5b'
device = 'cuda'

In [7]:
vit_backbone, model_transforms, _ = open_clip.create_model_and_transforms(model_name, pretrained=False)

mean, std = model_transforms.transforms[-1].mean, model_transforms.transforms[-1].std
image_size = model_transforms.transforms[0].size[0]

model = Model(vit_backbone.cpu()).to(device)

In [8]:
model.load_state_dict(torch.load('/mnt/SSD/workspace/atomic_hack/ViT-H-14-378-quickgelu_dfn5b_0.7920078574145561.pth'))

<All keys matched successfully>

In [9]:
model.eval()

Model(
  (vit_backbone): CLIP(
    (visual): VisionTransformer(
      (conv1): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (patch_dropout): Identity()
      (ln_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): ModuleList(
          (0-31): 32 x ResidualAttentionBlock(
            (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=1280, out_features=1280, bias=True)
            )
            (ls_1): Identity()
            (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=1280, out_features=5120, bias=True)
              (gelu): QuickGELU()
              (c_proj): Linear(in_features=5120, out_features=1280, bias=True)
            )
            (ls_2): Identity()
          )
        )
      

In [10]:
augs = A.Compose([
            A.Resize(image_size, image_size),
            A.Normalize(mean=mean, std=std, p=1)
        ])

In [17]:
fname = '/mnt/SSD/workspace/atomic_hack/Сварные_швы/dataset/5 (195).jpg'

In [18]:
pred_bboxes = yolo_model(fname, conf=0.05, iou=0.3, augment=True)


image 1/1 /mnt/SSD/workspace/atomic_hack/Сварные_швы/dataset/5 (195).jpg: 576x1024 5 прилегающие дефектыs, 30.8ms
Speed: 14.0ms preprocess, 30.8ms inference, 0.8ms postprocess per image at shape (1, 3, 576, 1024)


In [19]:
img = pred_bboxes[0].orig_img
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h, w, _ = img.shape

In [36]:
def plot_one_box(x, image, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(
        0.002 * (image.shape[0] + image.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(image, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(image, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(image, label, (c1[0], c1[1] - 2), cv2.FONT_HERSHEY_COMPLEX, tl / 3,
                    [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

In [37]:
idx2label = {
    0: 'прилегающие дефекты', 
    1: 'дефекты целостности',
    2: 'дефекты геометрии',
    3: 'дефекты постобработки',
    4: 'дефекты невыполнения'
}

In [38]:
if len(pred_bboxes[0].boxes.cls)==0:
    print('Нет деффектов')
else:
    imgs = []
    for bbox in pred_bboxes[0].boxes.xyxy:
        x1, y1, x2, y2 = bbox.cpu().tolist()
        x1_ = int(x1 // 1.2)
        y1_ = int(y1 // 1.2)
        x2_ = int(x2 * 1.2)
        y2_ = int(y2 * 1.2)       
        crop = img[y1_:y2_, x1_:x2_, :]
        crop = augs(image=crop)['image']
        crop = torch.from_numpy(crop).permute(2, 0, 1)
        imgs.append(crop)
    batch = torch.stack(imgs)
    
    with torch.no_grad():
        preds = model(batch.to(device))[0]
        preds = preds.cpu().detach().numpy().argmax(1)

    for bbox, pred in zip(pred_bboxes[0].boxes.xyxy, preds):
        plot_one_box(bbox.cpu().tolist(), img, color=(255, 0, 0), label=idx2label[pred])

    
    ans = {}
    for idx, bbox in enumerate(pred_bboxes[0].boxes.xywh):
        rel_x, rel_y, w_bbox, h_bbox = bbox.cpu().tolist()
        ans[idx] = {
            'filename': os.path.basename(fname),
            'rel_x': rel_x / w,
            'rel_y': rel_y / h,
            'w': w_bbox / w,
            'h': h_bbox / h,
            'class_id': preds[idx],
            
        }
    ans['img'] = img

In [110]:
python3.11 -m venv myvenv
source activate myvenv/bin/activate
pip install -r requirements.txt
python python src/main.py

In [16]:
dataset_files = glob.glob('/mnt/SSD/workspace/atomic_hack/Сварные_швы/dataset/*jpg')

ans = {}
idx = 0
for fname in tqdm(dataset_files):
    pred_bboxes = yolo_model(fname, conf=0.05, iou=0.3, augment=True)
    img = pred_bboxes[0].orig_img
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w, _ = img.shape

    imgs = []
    for bbox in pred_bboxes[0].boxes.xyxy:
        x1, y1, x2, y2 = bbox.cpu().tolist()
        x1_ = int(x1 // 1.2)
        y1_ = int(y1 // 1.2)
        x2_ = int(x2 * 1.2)
        y2_ = int(y2 * 1.2)       
        crop = img[y1_:y2_, x1_:x2_, :]
        crop = augs(image=crop)['image']
        crop = torch.from_numpy(crop).permute(2, 0, 1)
        imgs.append(crop)
    if not imgs:
        continue
    batch = torch.stack(imgs)
    
    with torch.no_grad():
        preds = model(batch.to(device))[0]
        preds = preds.cpu().detach().numpy().argmax(1)

    for i, bbox in enumerate(pred_bboxes[0].boxes.xywh):
        rel_x, rel_y, w_bbox, h_bbox = bbox.cpu().tolist()
        ans[idx] = {
            'filename': os.path.basename(fname),
            'rel_x': rel_x / w,
            'rel_y': rel_y / h,
            'width': w_bbox / w,
            'height': h_bbox / h,
            'class_id': preds[i],
            
        }
        idx += 1

submission = pd.DataFrame(ans).T

  0%|                                                  | 0/1162 [00:00<?, ?it/s]


image 1/1 /mnt/SSD/workspace/atomic_hack/Сварные_швы/dataset/8 (63).jpg: 1024x576 2 прилегающие дефектыs, 28.7ms
Speed: 16.8ms preprocess, 28.7ms inference, 0.8ms postprocess per image at shape (1, 3, 1024, 576)


  0%|                                          | 1/1162 [00:00<04:53,  3.95it/s]


image 1/1 /mnt/SSD/workspace/atomic_hack/Сварные_швы/dataset/11 (79).jpg: 1024x576 1 прилегающие дефекты, 24.5ms
Speed: 3.4ms preprocess, 24.5ms inference, 1.4ms postprocess per image at shape (1, 3, 1024, 576)


  0%|                                          | 2/1162 [00:00<03:42,  5.21it/s]

...

100%|██████████████████████████████████████▉| 1160/1162 [07:41<00:00,  2.36it/s]


image 1/1 /mnt/SSD/workspace/atomic_hack/Сварные_швы/dataset/3 (4).jpg: 576x1024 6 прилегающие дефектыs, 21.4ms
Speed: 2.9ms preprocess, 21.4ms inference, 1.1ms postprocess per image at shape (1, 3, 576, 1024)


100%|██████████████████████████████████████▉| 1161/1162 [07:42<00:00,  2.20it/s]


image 1/1 /mnt/SSD/workspace/atomic_hack/Сварные_швы/dataset/6 (56).jpg: 1024x576 3 прилегающие дефектыs, 20.2ms
Speed: 2.9ms preprocess, 20.2ms inference, 0.8ms postprocess per image at shape (1, 3, 1024, 576)


100%|███████████████████████████████████████| 1162/1162 [07:42<00:00,  2.51it/s]


In [22]:
submission.to_csv('submission.csv', index=False, sep=';')

In [26]:
submission

,filename,rel_x,rel_y,width,height,class_id
0,8 (63).jpg,0.525726,0.086109,0.211131,0.048237,2
1,8 (63).jpg,0.517117,0.676178,0.228122,0.107311,2
2,11 (79).jpg,0.463687,0.519255,0.177185,0.10945,2
3,6 (94).jpg,0.585802,0.227488,0.176822,0.075776,2
4,6 (94).jpg,0.468838,0.154408,0.113924,0.073892,2
...,...,...,...,...,...,...
5528,3 (4).jpg,0.472925,0.309481,0.176745,0.081906,4
5529,3 (4).jpg,0.993062,0.408936,0.013579,0.014656,3
5530,6 (56).jpg,0.583741,0.431398,0.189504,0.099235,2
5531,6 (56).jpg,0.445956,0.311311,0.153881,0.123304,2
